#### Named Entity Recognition in Clinical Notes using spaCy and Med7

Named Entity Recognition (`NER`) is a subtask of Natural Language Processing (`NLP`) that involves identifying and classifying named entities in unstructured data/text into predefined categories. 

`spaCy` is library for advanced NLP in Python. It comes with pretrained pipelines and features neural network models for tagging, parsing, named entity recognition, text classification and more.

`Med7` is a transferable clinical NLP processing model compatible with spaCy for `clinical` NER tasks. The `en_core_med7_lg` model is trained to recognize 7 categories: `DRUG`, `DOSAGE`, `STRENGTH`, `ROUTE`, `FREQUENCY`, `DURATION`, `FORM`, and is hosted on Huggingface.

Install model

In [0]:
%pip install https://huggingface.co/kormilitzin/en_core_med7_lg/resolve/main/en_core_med7_lg-any-py3-none-any.whl

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/607.4 MB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.6/607.4 MB 18.2 MB/s eta 0:00:34
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/607.4 MB 59.4 MB/s eta 0:00:11
     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/607.4 MB 74.6 MB/s eta 0:00:09
     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/607.4 MB 101.6 MB/s eta 0:00:06
     ╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/607.4 MB 107.6 MB/s eta 0:00:06
     ━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.2/607.4 MB 100.8 MB/s eta 0:00:06
     ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/607.4 MB 89.0 MB/s eta 0:00:07
     ━╸━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.3/607.4 MB 88.2 MB/s eta 0:00:07
     ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/607.4 MB 110.8 MB/s eta 0:00:06
     ━━╺━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.9/607.4 MB 111.3 MB/s eta 0:

In [0]:
%restart_python

Load model

In [0]:
import spacy

med7 = spacy.load("en_core_med7_lg")

# create distinct colours for labels
col_dict = {}
seven_colours = ['#e6194B', '#3cb44b', '#ffe119', '#ffd8b1', '#f58231', '#f032e6', '#42d4f4']
for label, colour in zip(med7.pipe_labels['ner'], seven_colours):
    col_dict[label] = colour

options = {'ents': med7.pipe_labels['ner'], 'colors':col_dict}

2024-06-25 03:29:42.468841: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Ingest data from AWS S3 into Databricks

In [0]:
import os
from pyspark.context import SparkContext 
from pyspark import SparkConf

In [0]:
data_path = '/FileStore/onco/data/'
os.environ['data_path'] = f'/dbfs{data_path}'
delta_path = '/mnt/onco/delta'

In [0]:
%sh
mkdir -p $data_path
cd $data_path
wget https://hls-eng-data-public.s3.amazonaws.com/data/mt_onc_50.zip -P $data_path
unzip -o mt_onc_50.zip

--2024-06-25 03:29:50--  https://hls-eng-data-public.s3.amazonaws.com/data/mt_onc_50.zip
Resolving hls-eng-data-public.s3.amazonaws.com (hls-eng-data-public.s3.amazonaws.com)... 52.217.72.188, 52.217.122.241, 3.5.6.124, ...
Connecting to hls-eng-data-public.s3.amazonaws.com (hls-eng-data-public.s3.amazonaws.com)|52.217.72.188|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 93586 (91K) [application/zip]
Saving to: ‘/dbfs/FileStore/onco/data/mt_onc_50.zip.1’

     0K .......... .......... .......... .......... .......... 54%  773K 0s
    50K .......... .......... .......... .......... .         100%  641K=0.1s

2024-06-25 03:29:51 (707 KB/s) - ‘/dbfs/FileStore/onco/data/mt_onc_50.zip.1’ saved [93586/93586]



Archive:  mt_onc_50.zip
  inflating: mt_onc_50/mt_oncology_0.txt  
  inflating: mt_onc_50/mt_oncology_1.txt  
  inflating: mt_onc_50/mt_oncology_10.txt  
  inflating: mt_onc_50/mt_oncology_11.txt  
  inflating: mt_onc_50/mt_oncology_12.txt  
  inflating: mt_onc_50/mt_oncology_13.txt  
  inflating: mt_onc_50/mt_oncology_14.txt  
  inflating: mt_onc_50/mt_oncology_15.txt  
  inflating: mt_onc_50/mt_oncology_16.txt  
  inflating: mt_onc_50/mt_oncology_17.txt  
  inflating: mt_onc_50/mt_oncology_18.txt  
  inflating: mt_onc_50/mt_oncology_19.txt  
  inflating: mt_onc_50/mt_oncology_2.txt  
  inflating: mt_onc_50/mt_oncology_20.txt  
  inflating: mt_onc_50/mt_oncology_21.txt  
  inflating: mt_onc_50/mt_oncology_22.txt  
  inflating: mt_onc_50/mt_oncology_23.txt  
  inflating: mt_onc_50/mt_oncology_24.txt  
  inflating: mt_onc_50/mt_oncology_25.txt  
  inflating: mt_onc_50/mt_oncology_26.txt  
  inflating: mt_onc_50/mt_oncology_27.txt  
  inflating: mt_onc_50/mt_oncology_28.txt  
  inflating

In [0]:
ls /dbfs/FileStore/onco/data/mt_onc_50/

mt_oncology_0.txt   mt_oncology_20.txt  mt_oncology_32.txt  mt_oncology_44.txt
mt_oncology_1.txt   mt_oncology_21.txt  mt_oncology_33.txt  mt_oncology_45.txt
mt_oncology_10.txt  mt_oncology_22.txt  mt_oncology_34.txt  mt_oncology_46.txt
mt_oncology_11.txt  mt_oncology_23.txt  mt_oncology_35.txt  mt_oncology_47.txt
mt_oncology_12.txt  mt_oncology_24.txt  mt_oncology_36.txt  mt_oncology_48.txt
mt_oncology_13.txt  mt_oncology_25.txt  mt_oncology_37.txt  mt_oncology_49.txt
mt_oncology_14.txt  mt_oncology_26.txt  mt_oncology_38.txt  mt_oncology_5.txt
mt_oncology_15.txt  mt_oncology_27.txt  mt_oncology_39.txt  mt_oncology_6.txt
mt_oncology_16.txt  mt_oncology_28.txt  mt_oncology_4.txt   mt_oncology_7.txt
mt_oncology_17.txt  mt_oncology_29.txt  mt_oncology_40.txt  mt_oncology_8.txt
mt_oncology_18.txt  mt_oncology_3.txt   mt_oncology_41.txt  mt_oncology_9.txt
mt_oncology_19.txt  mt_oncology_30.txt  mt_oncology_42.txt
mt_oncology_2.txt   mt_oncology_31.txt  mt_oncology_43.txt


In [0]:
dbutils.fs.cp("file:/dbfs/FileStore/onco/data/mt_onc_50/", "dbfs:/FileStore/onco", True)

True

In [0]:
sc = SparkContext.getOrCreate(SparkConf())
sdf = sc.wholeTextFiles("/FileStore/onco/").toDF().withColumnRenamed('_1', 'path').withColumnRenamed('_2', 'text')

In [0]:
display(sdf.limit(5))

path text dbfs:/FileStore/onco/mt_oncology_0.txt Sample Type / Medical Specialty:
Hematology - Oncology
Sample Name:
Discharge Summary - Mesothelioma - 1
Description:
Mesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.
(Medical Transcription Sample Report)
PRINCIPAL DIAGNOSIS:
Mesothelioma.
SECONDARY DIAGNOSES:
Pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.
PROCEDURES
1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural fluoroscopy.
2. On August 20, 2007, thoracentesis.
3. On August 31, 2007, Port-A-Cath placement.
HISTORY AND PHYSICAL:
The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion.
PAST MEDICAL HISTORY
1. Pericardectomy.
2. Pericarditis.
2. Atrial fibrillation.
4. RNCA with intracranial thrombolytic treatment.
5 PTA of MCA.
6. Mesenteric venous thrombosis.
7. Pericardial window.
8. Cholecystectomy.
9. Left thoracentesis.
FAMILY HISTORY:
No family history of coronary artery disease, CVA, diabetes, CHF or MI. The patient has one family member, a sister, with history of cancer.
SOCIAL HISTORY:
She is married. Employed with the US Post Office. She is a mother of three. Denies tobacco, alcohol or illicit drug use.
MEDICATIONS
1. Coumadin 1 mg daily. Last INR was on Tuesday, August 14, 2007, and her INR was 2.3.
2. Amiodarone 100 mg p.o. daily.
REVIEW OF SYSTEMS:
Complete review of systems negative except as in pulmonary as noted above. The patient also reports occasional numbness and tingling of her left arm.
PHYSICAL EXAMINATION
VITAL SIGNS: Blood pressure 123/95, heart rate 83, respirations 20, temperature 97, and oxygen saturation 97%.
GENERAL: Positive nonproductive cough and pain with coughing.
HEENT: Pupils are equal and reactive to light and accommodation. Tympanic membranes are clear.
NECK: Supple. No lymphadenopathy. No masses.
RESPIRATORY: Pleural friction rub is noted.
GI: Soft, nondistended, and nontender. Positive bowel sounds. No organomegaly.
EXTREMITIES: No edema, no clubbing, no cyanosis, no tenderness. Full range of motion. Normal pulses in all extremities.
SKIN: No breakdown or lesions. No ulcers.
NEUROLOGIC: Grossly intact. No focal deficits. Awake, alert, and oriented to person, place, and time.
LABORATORY DATA:
Labs are pending.
HOSPITAL COURSE:
The patient was admitted for a right-sided pleural effusion for thoracentesis on Monday by Dr. X. Her Coumadin was placed on hold. A repeat echocardiogram was checked. She was started on prophylaxis for DVT with Lovenox 40 mg subcutaneously. Her history dated back to March 2005 when she first sought medical attention for evidence of pericarditis, which was treated with pericardial window in an outside hospital, at that time she was also found to have mesenteric pain and thrombosis, is now anticoagulated. Her pericardial fluid was accumulated and she was seen by Dr. Y. At that time, she was recommended for pericardectomy, which was performed by Dr. Z. Review of her CT scan from March 2006 prior to her pericardectomy, already shows bilateral plural effusions. The patient improved clinically after the pericardectomy with resolution of her symptoms. Recently, she was readmitted to the hospital with chest pain and found to have bilateral pleural effusion, the right greater than the left. CT of the chest also revealed a large mediastinal lymph node. We reviewed the pathology obtained from the pericardectomy in March 2006, which was diagnostic of mesothelioma. At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid biopsies, whi

In [0]:
sdf.limit(1).select("text").collect()[0]

Row(text='Sample Type / Medical Specialty:\nHematology - Oncology\nSample Name:\nDischarge Summary - Mesothelioma - 1\nDescription:\nMesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.\n(Medical Transcription Sample Report)\nPRINCIPAL DIAGNOSIS:\nMesothelioma.\nSECONDARY DIAGNOSES:\nPleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.\nPROCEDURES\n1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural fluoroscopy.\n2. On August 20, 2007, thoracentesis.\n3. On August 31, 2007, Port-A-Cath placement.\nHISTORY AND PHYSICAL:\nThe patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and 

Write to delta bronze layer

In [0]:
sdf.write.format('delta').mode('overwrite').save(f'{delta_path}/bronze/onco')
display(dbutils.fs.ls(f'{delta_path}/bronze/onco'))

path,name,size,modificationTime
dbfs:/mnt/onco/delta/bronze/onco/_delta_log/,_delta_log/,0,0
dbfs:/mnt/onco/delta/bronze/onco/part-00000-08839211-456d-4089-9a8e-9b8155c7beb6-c000.snappy.parquet,part-00000-08839211-456d-4089-9a8e-9b8155c7beb6-c000.snappy.parquet,51510,1719015563000
dbfs:/mnt/onco/delta/bronze/onco/part-00000-157d95cd-e76a-4377-a000-cd1563b0a28c-c000.snappy.parquet,part-00000-157d95cd-e76a-4377-a000-cd1563b0a28c-c000.snappy.parquet,51510,1719286213000
dbfs:/mnt/onco/delta/bronze/onco/part-00000-1de56b7d-1dfd-4ed0-bba1-768d0e496f1d-c000.snappy.parquet,part-00000-1de56b7d-1dfd-4ed0-bba1-768d0e496f1d-c000.snappy.parquet,51616,1714270076000
dbfs:/mnt/onco/delta/bronze/onco/part-00000-32764967-9e09-419d-9b02-bcf7f1a60b91-c000.snappy.parquet,part-00000-32764967-9e09-419d-9b02-bcf7f1a60b91-c000.snappy.parquet,51510,1719010311000
dbfs:/mnt/onco/delta/bronze/onco/part-00000-35ece399-35ab-4091-b9a8-c1c20aba6e3d-c000.snappy.parquet,part-00000-35ece399-35ab-4091-b9a8-c1c20aba6e3d-c000.snappy.parquet,51510,1717552861000
dbfs:/mnt/onco/delta/bronze/onco/part-00000-377455f0-6e4f-469f-951d-b835ad95ef65-c000.snappy.parquet,part-00000-377455f0-6e4f-469f-951d-b835ad95ef65-c000.snappy.parquet,51510,1717630897000
dbfs:/mnt/onco/delta/bronze/onco/part-00000-656ac65c-0e29-41e8-96e2-1b542a03408f-c000.snappy.parquet,part-00000-656ac65c-0e29-41e8-96e2-1b542a03408f-c000.snappy.parquet,51510,1717983264000
dbfs:/mnt/onco/delta/bronze/onco/part-00000-684c2395-123c-4bc1-9dc5-3bad645dbd8c-c000.snappy.parquet,part-00000-684c2395-123c-4bc1-9dc5-3bad645dbd8c-c000.snappy.parquet,51510,1717982426000
dbfs:/mnt/onco/delta/bronze/onco/part-00000-ab287ca9-16dd-495c-9381-bf1dea3fdc12-c000.snappy.parquet,part-00000-ab287ca9-16dd-495c-9381-bf1dea3fdc12-c000.snappy.parquet,51510,1717714055000


In [0]:
%sql

SELECT * from delta.`/mnt/onco/delta/bronze/onco/`;

path text dbfs:/FileStore/onco/mt_oncology_0.txt Sample Type / Medical Specialty:
Hematology - Oncology
Sample Name:
Discharge Summary - Mesothelioma - 1
Description:
Mesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.
(Medical Transcription Sample Report)
PRINCIPAL DIAGNOSIS:
Mesothelioma.
SECONDARY DIAGNOSES:
Pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.
PROCEDURES
1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural fluoroscopy.
2. On August 20, 2007, thoracentesis.
3. On August 31, 2007, Port-A-Cath placement.
HISTORY AND PHYSICAL:
The patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to an urgent care center. Chest x-ray revealed right-sided pleural effusion.
PAST MEDICAL HISTORY
1. Pericardectomy.
2. Pericarditis.
2. Atrial fibrillation.
4. RNCA with intracranial thrombolytic treatment.
5 PTA of MCA.
6. Mesenteric venous thrombosis.
7. Pericardial window.
8. Cholecystectomy.
9. Left thoracentesis.
FAMILY HISTORY:
No family history of coronary artery disease, CVA, diabetes, CHF or MI. The patient has one family member, a sister, with history of cancer.
SOCIAL HISTORY:
She is married. Employed with the US Post Office. She is a mother of three. Denies tobacco, alcohol or illicit drug use.
MEDICATIONS
1. Coumadin 1 mg daily. Last INR was on Tuesday, August 14, 2007, and her INR was 2.3.
2. Amiodarone 100 mg p.o. daily.
REVIEW OF SYSTEMS:
Complete review of systems negative except as in pulmonary as noted above. The patient also reports occasional numbness and tingling of her left arm.
PHYSICAL EXAMINATION
VITAL SIGNS: Blood pressure 123/95, heart rate 83, respirations 20, temperature 97, and oxygen saturation 97%.
GENERAL: Positive nonproductive cough and pain with coughing.
HEENT: Pupils are equal and reactive to light and accommodation. Tympanic membranes are clear.
NECK: Supple. No lymphadenopathy. No masses.
RESPIRATORY: Pleural friction rub is noted.
GI: Soft, nondistended, and nontender. Positive bowel sounds. No organomegaly.
EXTREMITIES: No edema, no clubbing, no cyanosis, no tenderness. Full range of motion. Normal pulses in all extremities.
SKIN: No breakdown or lesions. No ulcers.
NEUROLOGIC: Grossly intact. No focal deficits. Awake, alert, and oriented to person, place, and time.
LABORATORY DATA:
Labs are pending.
HOSPITAL COURSE:
The patient was admitted for a right-sided pleural effusion for thoracentesis on Monday by Dr. X. Her Coumadin was placed on hold. A repeat echocardiogram was checked. She was started on prophylaxis for DVT with Lovenox 40 mg subcutaneously. Her history dated back to March 2005 when she first sought medical attention for evidence of pericarditis, which was treated with pericardial window in an outside hospital, at that time she was also found to have mesenteric pain and thrombosis, is now anticoagulated. Her pericardial fluid was accumulated and she was seen by Dr. Y. At that time, she was recommended for pericardectomy, which was performed by Dr. Z. Review of her CT scan from March 2006 prior to her pericardectomy, already shows bilateral plural effusions. The patient improved clinically after the pericardectomy with resolution of her symptoms. Recently, she was readmitted to the hospital with chest pain and found to have bilateral pleural effusion, the right greater than the left. CT of the chest also revealed a large mediastinal lymph node. We reviewed the pathology obtained from the pericardectomy in March 2006, which was diagnostic of mesothelioma. At this time, chest tube placement for drainage of the fluid occurred and thoracoscopy with fluid biopsies, whi

In [0]:
sample_text = sdf.limit(1).select("text").collect()[0]
sample_text.text

'Sample Type / Medical Specialty:\nHematology - Oncology\nSample Name:\nDischarge Summary - Mesothelioma - 1\nDescription:\nMesothelioma, pleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.\n(Medical Transcription Sample Report)\nPRINCIPAL DIAGNOSIS:\nMesothelioma.\nSECONDARY DIAGNOSES:\nPleural effusion, atrial fibrillation, anemia, ascites, esophageal reflux, and history of deep venous thrombosis.\nPROCEDURES\n1. On August 24, 2007, decortication of the lung with pleural biopsy and transpleural fluoroscopy.\n2. On August 20, 2007, thoracentesis.\n3. On August 31, 2007, Port-A-Cath placement.\nHISTORY AND PHYSICAL:\nThe patient is a 41-year-old Vietnamese female with a nonproductive cough that started last week. She has had right-sided chest pain radiating to her back with fever starting yesterday. She has a history of pericarditis and pericardectomy in May 2006 and developed cough with right-sided chest pain, and went to a

Perform NER with Med7

In [0]:
text = sample_text.text
doc = med7(text)

Visualize NERs

In [0]:
spacy.displacy.render(doc, style='ent', jupyter=True, options=options)

In [0]:
import numpy as np
import pandas as pd

entities = [(ent.text, ent.label_, ent.start, ent.end) for ent in doc.ents]
entities_df = pd.DataFrame(entities, columns=['Chunk', 'Entity', 'Start', 'End'])
entities_df

,Chunk,Entity,Start,End
0,Coumadin,DRUG,350,351
1,1 mg,STRENGTH,351,353
2,daily,FREQUENCY,353,354
3,Amiodarone,DRUG,375,376
4,100 mg,STRENGTH,376,378
5,p.o.,ROUTE,378,380
6,daily,FREQUENCY,380,381
7,Coumadin,DRUG,601,602
8,Lovenox,DRUG,621,622
9,40 mg,STRENGTH,622,624


In [0]:
import plotly.express as px

fig = px.scatter(entities_df, x="Start", y="End", color="Entity", hover_data=["Chunk"])
fig.update_layout(title="Named Entities in Clinical Note")
fig.show()


Further processing

In [0]:
clinical_notes  = spark.sql("SELECT path, text from delta.`/mnt/onco/delta/bronze/onco/`").toPandas()

In [0]:
clinical_notes_nlp = {clinical_notes.path[i]:med7(clinical_notes.text[i]) for i in range(len(clinical_notes))}

In [0]:
#for i in range(len(clinical_notes_nlp)):
#    spacy.displacy.render(clinical_notes_nlp[f'dbfs:/FileStore/onco/mt_oncology_{i}.txt'], style='ent', jupyter= True,  options=options)

In [0]:
ner_pdf = pd.DataFrame()
for i in range(len(clinical_notes_nlp)):
    path = f"dbfs:/FileStore/onco/mt_oncology_{i}.txt"    
    named_entities = {e.text: e.label_ for e in clinical_notes_nlp[path].ents}    
    ner_pdf_i = pd.DataFrame(named_entities.items(), columns=['chunk', 'entity'])
    ner_pdf_i.insert(loc=0, column='path', value=path)    
    ner_pdf = pd.concat([ner_pdf, ner_pdf_i], axis=0)

In [0]:
ner_pdf.head(22)

,path,chunk,entity
0,dbfs:/FileStore/onco/mt_oncology_0.txt,Coumadin,DRUG
1,dbfs:/FileStore/onco/mt_oncology_0.txt,1 mg,STRENGTH
2,dbfs:/FileStore/onco/mt_oncology_0.txt,daily,FREQUENCY
3,dbfs:/FileStore/onco/mt_oncology_0.txt,Amiodarone,DRUG
4,dbfs:/FileStore/onco/mt_oncology_0.txt,100 mg,STRENGTH
5,dbfs:/FileStore/onco/mt_oncology_0.txt,p.o.,ROUTE
6,dbfs:/FileStore/onco/mt_oncology_0.txt,Lovenox,DRUG
7,dbfs:/FileStore/onco/mt_oncology_0.txt,40 mg,STRENGTH
8,dbfs:/FileStore/onco/mt_oncology_0.txt,subcutaneously,ROUTE
9,dbfs:/FileStore/onco/mt_oncology_0.txt,chemotherapy,DRUG


In [0]:
from collections import Counter

drug_entities = dict([e for e in Counter(list(ner_pdf[ner_pdf.entity == "DRUG"]["chunk"])).most_common() if e[1] >= 6])
drug_entities_df = pd.DataFrame(drug_entities.items(), columns=['Drug', 'Frequency'])
drug_entities_df

,Drug,Frequency
0,chemotherapy,12
1,ANESTHESIA,9
2,Coumadin,6
3,Vicodin,6
4,antibiotics,6
5,carboplatin,6


In [0]:
px.bar(drug_entities_df, x='Drug', y = 'Frequency', title = "Frequency of drugs in clinical notes")

#### RxNorm coding for drug entities extracted from clinical notes

RxNorm is a standardized nomenclature for clinical drugs produced by the U.S. National Library of Medicine (NLM). RxNorm represents one of a suite of designated standards for use in U.S. Federal government systems for the electronic exchange of clinical health information.

The goal of RxNorm is to allow various systems using different drug nomenclatures to share data efficiently.

RxNorm provides standard names for clinical drugs (active ingredient + strength + dose form) and for dose forms as administered to a patient. It provides links from clinical drugs, both branded and generic, to their active ingredients, drug components (active ingredient + strength), and related brand names.

In [0]:
base_url = "https://rxnav.nlm.nih.gov/REST"
drug_ents = ["Coumadin", "Amiodarone", "Lovenox", "cisplatin"]
rxcuis = []

In [0]:
%pip install xmltodict

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.
Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


In [0]:
import requests
import json
import xmltodict
import numpy as np
import pandas as pd

In [0]:
for drug_ent in drug_ents:
    # Construct the API request URL
    request_url = f"{base_url}/drugs?name={''.join(drug_ent)}"   

    # Send the API request and retrieve the response
    response = requests.get(request_url)

    # Convert response XML to JSON
    response_json_dict = xmltodict.parse(response.content)
    response_json_str = json.dumps(response_json_dict['rxnormdata'])
    response_json = json.loads(response_json_str)

    # Extract the RxCUI and concept name    
    for group in response_json["drugGroup"]["conceptGroup"]:       
        if "conceptProperties" in group.keys():
            concept = group["conceptProperties"]   
            if isinstance(concept, dict):
                rxcui = group["conceptProperties"]["rxcui"]
                name = group["conceptProperties"]["name"] 
                rxcuis.append([drug_ent, rxcui, name])   
            elif isinstance(concept, list):
                for i in range(len(concept)):
                    rxcui = group["conceptProperties"][i]["rxcui"]
                    name = group["conceptProperties"][i]["name"]       
                    rxcuis.append([drug_ent, rxcui, name])
    
# Create Pandas DataFrame from RxNorm data
rxnorm_pdf = pd.DataFrame(rxcuis, columns=['Drug', 'RxCUI', 'Concept'])

In [0]:
rxnorm_pdf

,Drug,RxCUI,Concept
0,Coumadin,855290,warfarin sodium 1 MG Oral Tablet [Coumadin]
1,Coumadin,855298,warfarin sodium 10 MG Oral Tablet [Coumadin]
2,Coumadin,855304,warfarin sodium 2 MG Oral Tablet [Coumadin]
3,Coumadin,855314,warfarin sodium 2.5 MG Oral Tablet [Coumadin]
4,Coumadin,855320,warfarin sodium 3 MG Oral Tablet [Coumadin]
5,Coumadin,855326,warfarin sodium 4 MG Oral Tablet [Coumadin]
6,Coumadin,855334,warfarin sodium 5 MG Oral Tablet [Coumadin]
7,Coumadin,855340,warfarin sodium 6 MG Oral Tablet [Coumadin]
8,Coumadin,855346,warfarin sodium 7.5 MG Oral Tablet [Coumadin]
9,Amiodarone,1046408,100 ML amiodarone hydrochloride 1.5 MG/ML Inje...


In [0]:
drug_ent = "Lovenox"

In [0]:
# Construct the API request URL
request_url = f"{base_url}/drugs?name={''.join(drug_ent)}"


In [0]:
# Send the API request and retrieve the response
response = requests.get(request_url)

In [0]:
response.headers

{'Date': 'Tue, 02 Jul 2024 23:53:19 GMT', 'Content-Type': 'application/xml; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Server': 'nginx', 'Vary': 'Accept', 'X-Download-Options': 'noopen', 'ETag': 'W/"20AA_240701F;1719412772;3.1.295"', 'X-Permitted-Cross-Domain-Policies': 'none', 'Cache-Control': 'public, max-age=300', 'Content-Security-Policy': "object-src 'none'; script-src 'unsafe-inline' 'unsafe-eval' 'strict-dynamic' https: http:;", 'Strict-Transport-Security': 'max-age=31622400; includeSubDomains; preload;', 'X-XSS-Protection': '1; mode=block', 'X-Content-Type-Options': 'nosniff', 'X-Frame-Options': 'DENY', 'X-Cache-Status': 'MISS', 'Access-Control-Allow-Origin': '*', 'Access-Control-Allow-Headers': '*', 'Content-Encoding': 'gzip'}

In [0]:
response.content

b'<?xml version="1.0" ?><rxnormdata><drugGroup><name></name><conceptGroup><tty>BPCK</tty></conceptGroup><conceptGroup><tty>SBD</tty><conceptProperties><rxcui>854232</rxcui><name>0.3 ML enoxaparin sodium 100 MG/ML Prefilled Syringe [Lovenox]</name><synonym>0.3 ML Lovenox 100 MG/ML Prefilled Syringe</synonym><tty>SBD</tty><language>ENG</language><suppress>N</suppress><umlscui></umlscui></conceptProperties><conceptProperties><rxcui>854236</rxcui><name>0.4 ML enoxaparin sodium 100 MG/ML Prefilled Syringe [Lovenox]</name><synonym>0.4 ML Lovenox 100 MG/ML Prefilled Syringe</synonym><tty>SBD</tty><language>ENG</language><suppress>N</suppress><umlscui></umlscui></conceptProperties><conceptProperties><rxcui>854239</rxcui><name>0.6 ML enoxaparin sodium 100 MG/ML Prefilled Syringe [Lovenox]</name><synonym>0.6 ML Lovenox 100 MG/ML Prefilled Syringe</synonym><tty>SBD</tty><language>ENG</language><suppress>N</suppress><umlscui></umlscui></conceptProperties><conceptProperties><rxcui>854242</rxcui><na

In [0]:
response_json_dict = xmltodict.parse(response.content)
response_json_str = json.dumps(response_json_dict['rxnormdata'])
response_json = json.loads(response_json_str)

In [0]:
response_json

{'drugGroup': {'name': None,
  'conceptGroup': [{'tty': 'BPCK'},
   {'tty': 'SBD',
    'conceptProperties': [{'rxcui': '854232',
      'name': '0.3 ML enoxaparin sodium 100 MG/ML Prefilled Syringe [Lovenox]',
      'synonym': '0.3 ML Lovenox 100 MG/ML Prefilled Syringe',
      'tty': 'SBD',
      'language': 'ENG',
      'suppress': 'N',
      'umlscui': None},
     {'rxcui': '854236',
      'name': '0.4 ML enoxaparin sodium 100 MG/ML Prefilled Syringe [Lovenox]',
      'synonym': '0.4 ML Lovenox 100 MG/ML Prefilled Syringe',
      'tty': 'SBD',
      'language': 'ENG',
      'suppress': 'N',
      'umlscui': None},
     {'rxcui': '854239',
      'name': '0.6 ML enoxaparin sodium 100 MG/ML Prefilled Syringe [Lovenox]',
      'synonym': '0.6 ML Lovenox 100 MG/ML Prefilled Syringe',
      'tty': 'SBD',
      'language': 'ENG',
      'suppress': 'N',
      'umlscui': None},
     {'rxcui': '854242',
      'name': '0.8 ML enoxaparin sodium 100 MG/ML Prefilled Syringe [Lovenox]',
      'syno

In [0]:
# Extract the RxCUI and concept name
rxcuis = []
for group in response_json["drugGroup"]["conceptGroup"]:       
    if "conceptProperties" in group.keys():
        concept = group["conceptProperties"]   
        if isinstance(concept, dict):
            rxcui = group["conceptProperties"]["rxcui"]
            name = group["conceptProperties"]["name"] 
            rxcuis.append([drug_ent, rxcui, name])   
        elif isinstance(concept, list):
            for i in range(len(concept)):
                rxcui = group["conceptProperties"][i]["rxcui"]
                name = group["conceptProperties"][i]["name"]       
                rxcuis.append([drug_ent, rxcui, name])


In [0]:
rxnorm_pdf_lovenox = pd.DataFrame(rxcuis, columns=['Drug', 'RxCUI', 'Concept'])

In [0]:
rxnorm_pdf_lovenox

,Drug,RxCUI,Concept
0,Lovenox,854232,0.3 ML enoxaparin sodium 100 MG/ML Prefilled S...
1,Lovenox,854236,0.4 ML enoxaparin sodium 100 MG/ML Prefilled S...
2,Lovenox,854239,0.6 ML enoxaparin sodium 100 MG/ML Prefilled S...
3,Lovenox,854242,0.8 ML enoxaparin sodium 100 MG/ML Prefilled S...
4,Lovenox,854247,0.8 ML enoxaparin sodium 150 MG/ML Prefilled S...
5,Lovenox,854249,1 ML enoxaparin sodium 100 MG/ML Prefilled Syr...
6,Lovenox,854253,1 ML enoxaparin sodium 150 MG/ML Prefilled Syr...
7,Lovenox,854256,enoxaparin sodium 100 MG/ML Injectable Solutio...
